In [ ]:
%load_ext autoreload
%autoreload 2

import os

CURRENT_DPATH = os.path.abspath(os.path.dirname("__file__"))
ROOT_DPATH = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir))
DATA_DPATH = os.path.join(ROOT_DPATH, "data")

import pandas as pd 

from motor_defects_classification.utils.load import load_class_names
from motor_defects_classification.utils.fs import get_date_string

In [ ]:
def get_target_class(fpath: str) -> str:
    target_classes = load_class_names()
    for class_name in target_classes:
        if class_name in fpath.lower():
            return class_name

In [ ]:
data = []
for dir_path, _dir_name, file_names in os.walk(DATA_DPATH):
    for file_name in file_names:
        fname, fext = os.path.splitext(file_name)
        if fext != ".csv":
            continue

        full_fpath = os.path.join(dir_path, file_name)
        raw_df = pd.read_csv(full_fpath)
        # get the last time value
        duration = raw_df.Time.iat[-1]

        short_dpath = dir_path.split(DATA_DPATH)[-1]
        fpath = os.path.join(short_dpath, file_name)
        trg_name = get_target_class(fpath)
        data.append([fpath, duration, raw_df.shape[0], trg_name])

df = pd.DataFrame(data, columns=["fpath", "duration", "n_samples", "target"])

In [ ]:
df = pd.concat(
    [df, pd.get_dummies(df.target)], 
    axis=1
)

In [ ]:
df

In [ ]:
_SAVE = True

if _SAVE:
    save_dpath = os.path.join(DATA_DPATH, get_date_string())
    os.makedirs(save_dpath, exist_ok=True)

    df.to_excel(os.path.join(save_dpath, "train.xlsx"))